In [1]:
import os
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator

Retrieve Image Data

In [2]:
# TRAINING_DIR = os.path.join('lettuce', 'train')
# TESTING_DIR = os.path.join('lettuce', 'test')
# VALIDATION_DIR = os.path.join('lettuce', 'valid')
DIR = './FNNPK/'

Preprocessing Image

In [3]:
# train_datagen = ImageDataGenerator(rescale=1./255)
# test_datagen = ImageDataGenerator(rescale=1./255)
# valid_datagen = ImageDataGenerator(rescale=1./255)
# 
# train_generator = train_datagen.flow_from_directory(
#     TRAINING_DIR,
#     target_size=(300, 300),
#     batch_size=32,
#     class_mode='categorical'
# )
# testing_generator = test_datagen.flow_from_directory(
#     TESTING_DIR,
#     target_size=(300, 300),
#     batch_size=32,
#     class_mode='categorical'
# )
# validation_generator = valid_datagen.flow_from_directory(
#     VALIDATION_DIR,
#     target_size=(300, 300),
#     batch_size=32,
#     class_mode='categorical'
# )

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
valid_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
)
train_generator = train_datagen.flow_from_directory(
    DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
validation_generator = valid_datagen.flow_from_directory(
    DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 168 images belonging to 4 classes.
Found 40 images belonging to 4 classes.


Build a custom callback

In [4]:
class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self, monitor1, monitor2, target):
        super(CustomCallback, self).__init__()
        self.monitor1 = monitor1
        self.monitor2 = monitor2
        self.target = target
        
    def on_epoch_end(self, epoch, logs=None):
        current1 = logs[self.monitor1]
        current2 = logs[self.monitor2]
        if current1 is not None and current2 is not None:
            if current1 > self.target and current2 > self.target:
                print(f'{current1} and {current2} has reached {self.target}. Stopping training early.')
                self.model.stop_training = True
                
stop_when_80 = CustomCallback(monitor1='accuracy', monitor2='val_accuracy', target=0.85)

Build the model

In [15]:
# from tensorflow.keras.applications import VGG16

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(224, 224, 3)))

model.add(tf.keras.layers.Conv2D(16, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu'))


# Output
model.add(tf.keras.layers.Dense(4, activation='softmax'))

# Load the VGG16 model
# base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# 
# # Freeze the base model
# base_model.trainable = False
# 
# # Add custom layers on top
# x = base_model.output
# x = tf.keras.layers.Flatten()(x)
# x = tf.keras.layers.Dense(512, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.5)(x)
# x = tf.keras.layers.Dense(256, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.5)(x)
# x = tf.keras.layers.Dense(4, activation='softmax')(x)
# 
# model = tf.keras.models.Model(inputs=base_model.input, outputs=x)
# 
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 222, 222, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 111, 111, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 109, 109, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 52, 52, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 26, 26, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 43264)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │    22,151,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 4)              │         2,052 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,177,316 (84.60 MB)

 Trainable params: 22,177,316 (84.60 MB)

 Non-trainable params: 0 (0.00 B)

Compile the model

In [16]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

Train the model

In [17]:
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.1, 
    patience=3, 
    verbose=1
)

model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    callbacks=[stop_when_80]
)

Epoch 1/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 617ms/step - accuracy: 0.3554 - loss: 4.8343 - val_accuracy: 0.3500 - val_loss: 1.5554
Epoch 2/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 473ms/step - accuracy: 0.3978 - loss: 1.3657 - val_accuracy: 0.5000 - val_loss: 1.1025
Epoch 3/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 446ms/step - accuracy: 0.5464 - loss: 1.1428 - val_accuracy: 0.6750 - val_loss: 0.9585
Epoch 4/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 416ms/step - accuracy: 0.4914 - loss: 1.0511 - val_accuracy: 0.5750 - val_loss: 1.0443
Epoch 5/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 445ms/step - accuracy: 0.5924 - loss: 0.8906 - val_accuracy: 0.6250 - val_loss: 1.0696
Epoch 6/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 432ms/step - accuracy: 0.6989 - loss: 0.7873 - val_accuracy: 0.6250 - val_loss: 1.1247
Epoch 7/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 432ms/step - accuracy: 0.6878 - loss: 0.7747 - val_accuracy: 0.6750 - val_loss: 0.9757
Epoch 8/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 438ms/step - accuracy: 0.7009 - loss: 0.7070 - val_accuracy: 0.6250 - val_loss:

Evaluate the model

In [18]:
model.evaluate(validation_generator)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 150ms/step - accuracy: 0.8208 - loss: 0.5154


[0.49009791016578674, 0.824999988079071]